In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import datetime, time, gc
# from utils import distance, haversine, standard, pad_seq 
from scipy.stats import skew, kurtosis
# from catboost import CatBoostClassifier
from zipfile import ZipFile
from collections import Counter 
from sklearn.metrics import roc_auc_score as auc
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [2]:
# !pip install lightgbm
!free -m

              total        used        free      shared  buff/cache   available
Mem:          96166        8801       73983          18       13381       87201
Swap:             0           0           0


In [3]:
train_fea = pd.read_hdf("data/train_1kw_fea.hdf", 'df')
test_fea = pd.read_hdf("data/test_1kw_fea.hdf", 'df')  

In [4]:
uid_emb = pd.read_hdf("data/feature_uid_embed.hdf", 'df')
ua_pagerank = pd.read_hdf("data/feature_uq_pagerank.hdf", 'df')

In [5]:
uid_emb.head(), ua_pagerank.head()

(   uid  ProNE_Emb_0  ProNE_Emb_1  ProNE_Emb_2  ProNE_Emb_3  ProNE_Emb_4  \
 0    0    -0.312191    -0.204009     0.136454    -0.461219     0.009174   
 1    1    -0.381545    -0.066154     0.626009    -0.195843    -0.183346   
 2    2    -0.349376    -0.121073     0.362863    -0.275752    -0.153385   
 3    3    -0.360660    -0.217708     0.101080    -0.627966     0.024231   
 4    4    -0.232261     0.114976     0.487496    -0.141066    -0.369687   
 
    ProNE_Emb_5  ProNE_Emb_6  ProNE_Emb_7  ProNE_Emb_8  ProNE_Emb_9  \
 0    -0.157362     0.063997    -0.006948    -0.034682     0.144556   
 1    -0.164542    -0.083611    -0.048018     0.212018    -0.074528   
 2     0.372635     0.144914    -0.069478     0.050202     0.134235   
 3    -0.127040    -0.230191    -0.089313    -0.083376     0.058229   
 4    -0.134960     0.191774    -0.226353    -0.174449    -0.250017   
 
    ProNE_Emb_10  ProNE_Emb_11  ProNE_Emb_12  ProNE_Emb_13  ProNE_Emb_14  \
 0      0.085696      0.115629     -0.

In [6]:
train_fea = train_fea.merge(uid_emb, on='uid', how='left')
test_fea = test_fea.merge(uid_emb, on='uid', how='left')

In [7]:
train_fea = pd.concat([train_fea, ua_pagerank.iloc[:-1000000]], axis=1)
test_fea = pd.concat([test_fea, ua_pagerank.iloc[-1000000:]], axis=1)

In [8]:
train_fea.head()

,adv_id,adv_prim_id,age,app_first_class,app_score,app_second_class,career,city,city_rank,communication_avgonline_30d,communication_onlinerate,consume_purchase,creat_type_cd,dev_id,device_name,device_price,device_size,emui_dev,gender,his_app_size,his_on_shelf_time,id,indu_name,inter_type_cd,label,list_time,membership_life_duration,net_type,pt_d,residence,slot_id,spread_app_id,tags,task_id,uid,up_life_duration,up_membership_grade,communication_hours,uidtask_id_nunique,uidadv_id_nunique,uiddev_id_nunique,uidslot_id_nunique,uidadv_prim_id_nunique,uidspread_app_id_nunique,uidindu_name_nunique,agetask_id_nunique,ageadv_id_nunique,agedev_id_nunique,ageslot_id_nunique,ageadv_prim_id_nunique,agespread_app_id_nunique,ageindu_name_nunique,careertask_id_nunique,careeradv_id_nunique,careerdev_id_nunique,careerslot_id_nunique,careeradv_prim_id_nunique,careerspread_app_id_nunique,careerindu_name_nunique,citytask_id_nunique,cityadv_id_nunique,citydev_id_nunique,cityslot_id_nunique,cityadv_prim_id_nunique,cityspread_app_id_nunique,cityindu_name_nunique,adv_id_count_fea,adv_prim_id_count_fea,task_id_count_fea,uid_count_fea,city_count_fea,cnt_click_of_uid_and_task_id,cnt_click_of_uid_and_adv_id,cnt_click_of_uid_and_spread_app_id,cnt_click_of_uid_and_adv_prim_id,cnt_click_of_uid_and_dev_id,cnt_click_of_uid_and_app_second_class,cnt_click_of_age_and_task_id,cnt_click_of_age_and_adv_id,cnt_click_of_age_and_spread_app_id,cnt_click_of_age_and_adv_prim_id,cnt_click_of_age_and_dev_id,cnt_click_of_age_and_app_second_class,cnt_click_of_gender_and_task_id,cnt_click_of_gender_and_adv_id,cnt_click_of_gender_and_spread_app_id,cnt_click_of_gender_and_adv_prim_id,cnt_click_of_gender_and_dev_id,cnt_click_of_gender_and_app_second_class,cnt_click_of_city_and_task_id,cnt_click_of_city_and_adv_id,cnt_click_of_city_and_spread_app_id,cnt_click_of_city_and_adv_prim_id,cnt_click_of_city_and_dev_id,cnt_click_of_city_and_app_second_class,cnt_click_of_career_and_task_id,cnt_click_of_career_and_adv_id,cnt_click_of_career_and_spread_app_id,cnt_click_of_career_and_adv_prim_id,cnt_click_of_career_and_dev_id,cnt_click_of_career_and_app_second_class,uid_ctr,task_id_ctr,adv_id_ctr,city_ctr,spread_app_id_ctr,age_task_id_ctr,age_adv_id_ctr,age_spread_app_id_ctr,age_adv_prim_id_ctr,age_slot_id_ctr,city_task_id_ctr,city_adv_id_ctr,city_spread_app_id_ctr,city_adv_prim_id_ctr,city_slot_id_ctr,career_task_id_ctr,career_adv_id_ctr,career_spread_app_id_ctr,career_adv_prim_id_ctr,career_slot_id_ctr,net_type_target_enc,task_id_target_enc,adv_id_target_enc,adv_prim_id_target_enc,age_target_enc,app_first_class_target_enc,app_second_class_target_enc,career_target_enc,city_target_enc,consume_purchase_target_enc,uid_target_enc,dev_id_target_enc,tags_target_enc,slot_id_target_enc,ProNE_Emb_0,ProNE_Emb_1,ProNE_Emb_2,ProNE_Emb_3,ProNE_Emb_4,ProNE_Emb_5,ProNE_Emb_6,ProNE_Emb_7,ProNE_Emb_8,ProNE_Emb_9,ProNE_Emb_10,ProNE_Emb_11,ProNE_Emb_12,ProNE_Emb_13,ProNE_Emb_14,ProNE_Emb_15,ProNE_Emb_16,ProNE_Emb_17,ProNE_Emb_18,ProNE_Emb_19,ProNE_Emb_20,ProNE_Emb_21,ProNE_Emb_22,ProNE_Emb_23,ProNE_Emb_24,ProNE_Emb_25,ProNE_Emb_26,ProNE_Emb_27,ProNE_Emb_28,ProNE_Emb_29,ProNE_Emb_30,ProNE_Emb_31,pr_q,pr_a
0,2967,53,5,2,2,10,2,148,3,10,2116,0,5,40,69,5,162,20,0,14,3,NaN,6,3,0,4,-1,0,3,8,6,44,24,188,355482,18,0,18,14,14,6,4,8,6,6,4101,4975,56,12,107,74,38,4235,5145,56,12,107,74,38,1782,1824,45,12,87,62,30,0.142212,0.683594,0.142212,0.041473,0.032501,0,0,1,1,0,1,8,8,52,70,42,100,14,14,69,69,47,100,0,0,1,2,1,4,10,10,52,64,43,93,0.003431,0.042328,0.042328,0.034149,0.035217,0.024918,0.024918,0.025635,0.025635,0.026733,0.013557,0.013344,0.033936,0.033905,0.032532,0.031769,0.031769,0.026886,0.026886,0.028732,0.035919,0.041718,0.041718,0.035187,0.026154,0.038757,0.043579,0.028671,0.034790,0.035034,0.0,0.034332,0.044312,0.030518,-0.366386,-0.055409,0.348945,-0.214057,0.066230,0.146826,0.193919,0.097498,-0.148267,0.221713,-0.133683,0.039528,0.102285,0.078841,-0.205252,0.114743,0.106724,-0.155015,0.055775,-0.085198,-0.045131,

In [9]:
cate_cols = [
       'uid', 'task_id', 'adv_id', 'creat_type_cd', 'adv_prim_id',
       'dev_id', 'inter_type_cd', 'slot_id', 'spread_app_id', 'tags',
       'app_first_class', 'app_second_class', 'city',
       'device_name', 'career', 'gender', 'net_type',
       'residence', 'his_on_shelf_time', "communication_onlinerate", 
       'up_membership_grade', 'consume_purchase', 'indu_name',
]

use_cols = list( set(train_fea.columns) - set(['label', 'cnt', 'random_sector', 'pt_d',
        'uid_task_id_ctr', 'uid_adv_id_ctr', 'uid_spread_app_id_ctr', 'uid_adv_prim_id_ctr', 'uid_dev_id_ctr',
        'uid_app_second_class_ctr']))

print(len(use_cols), len(cate_cols))  

167 23


In [10]:
X_train = train_fea[train_fea["pt_d"]<=6].copy()
y_train = X_train["label"].astype('int32')
X_valid = train_fea[train_fea["pt_d"]>6]
y_valid = X_valid["label"].astype('int32')  

In [14]:
%%time


imp = pd.DataFrame()  # 特征重要性
imp['feat'] = use_cols

# params
params = {
    'learning_rate': 0.03,
    'objective': 'binary',
    'metric': ['auc'], 
    'boosting_type': 'gbdt',
    'num_leaves': 128,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.5,
    'bagging_freq': 5,
    'seed': 8,
    'bagging_seed': 1,
    'feature_fraction_seed': 7,
    'min_data_in_leaf': 20,
    'nthread': 30,
    'verbose': 1,
    }

# train_x, train_y = train[train['random_sector'] != 1][use_cols], train[train['random_sector'] != 1]['label'].values
# valid_x, valid_y = train[train['random_sector'] == 1][use_cols], train[train['random_sector'] == 1]['label'].values
# 数据加载
n_train = lgb.Dataset(X_train[use_cols], label=y_train, 
                             # free_raw_data=False
                     )
n_valid = lgb.Dataset(X_valid[use_cols], label=y_valid, 
                              #free_raw_data=False
                     )

clf = lgb.train(
    params=params,
    train_set=n_train,
    categorical_feature=cate_cols,
    num_boost_round=5000,
    valid_sets=[n_train, n_valid],
    early_stopping_rounds=50,
    verbose_eval=40,
)

val_pred = clf.predict(X_valid[use_cols], num_iteration=clf.best_iteration)

print("val_auc = {}".format(auc(y_valid, val_pred)))

test_pred = clf.predict(test_fea[use_cols], num_iteration=clf.best_iteration)

imp['gain'] = clf.feature_importance(importance_type='gain')
imp['split'] = clf.feature_importance(importance_type='split')
result = pd.DataFrame({
        'id': np.load("data/test_id.npy"), 
        'probability': test_pred, 
    })
# result.to_pickle("result/result1.pkl")
# imp.to_pickle("result/fea1.pkl")

/opt/conda/envs/python3/lib/python3.6/site-packages/lightgbm/basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['adv_id', 'adv_prim_id', 'app_first_class', 'app_second_class', 'career', 'city', 'communication_onlinerate', 'consume_purchase', 'creat_type_cd', 'dev_id', 'device_name', 'gender', 'his_on_shelf_time', 'indu_name', 'inter_type_cd', 'net_type', 'residence', 'slot_id', 'spread_app_id', 'tags', 'task_id', 'uid', 'up_membership_grade']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


Training until validation scores don't improve for 50 rounds
[40]	training's auc: 0.826336	valid_1's auc: 0.799817
[80]	training's auc: 0.836414	valid_1's auc: 0.801569
[120]	training's auc: 0.84443	valid_1's auc: 0.802065
[160]	training's auc: 0.850291	valid_1's auc: 0.802146
[200]	training's auc: 0.854723	valid_1's auc: 0.802176
[240]	training's auc: 0.858795	valid_1's auc: 0.802026
Early stopping, best iteration is:
[191]	training's auc: 0.853738	valid_1's auc: 0.802313
val_auc = 0.8023127445455208
CPU times: user 1h 30min 6s, sys: 25.7 s, total: 1h 30min 32s
Wall time: 6min 41s


In [15]:
result

,id,probability
0,1,0.005069
1,2,0.105597
2,3,0.135118
3,4,0.013243
4,5,0.072585
...,...,...
999995,999996,0.028653
999996,999997,0.027704
999997,999998,0.042450
999998,999999,0.008608


In [13]:
result.to_csv("submission.csv", index=False)   